Imported libraries

In [1]:
import pandas as pd

# For Data
from Data.historical_data import Historical_data
from alpaca.data.timeframe import TimeFrame
from alpaca.data.historical import CryptoHistoricalDataClient, StockHistoricalDataClient

# For preprocessing
from Preprocessing.preprocessing import Preprocessing

# For model
from Models.LSTM.lstm import LSTM

Data and Preprocessing

In [2]:
s_type = "StockBars"
stock = ["AAPL"]
timeframe = TimeFrame.Hour
start = "2021-08-19 06:00:00"
end = "2022-08-20 06:00:00"
client = StockHistoricalDataClient("PKV7BABG3DGE0GYL137W", "af1eQghpVgbyX2lh2T9n2ColmVYXHCWndQbKR3Lr")
save_csv = False

df = Historical_data(s_type = s_type, stock = stock, timeframe = timeframe, start = start, end = end, client = client, save_csv = save_csv)
df_tuple = Preprocessing(df)

Train model

In [3]:

num_epochs = 1000 #1000 epochs
learning_rate = 0.001 #0.001 lr

input_size = 7 #number of features
hidden_size = 2 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = 1 #number of output classes 

lstm = LSTM(num_classes, input_size, hidden_size, num_layers, df_tuple[0].shape[1]) #our lstm class 




NameError: name 'LSTM1' is not defined